In [ ]:
"""
@author: mdigi14
"""

import requests
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import time

"""
Update Parameters Here
"""
COLLECTION = "MekaVerse"
CONTRACT = "0x9A534628B4062E123cE7Ee2222ec20B86e16Ca8F"
YEAR = "2021"
MONTH = "10"
DAY = "13"
HOUR = "15"
MINUTE = "00"
REVEAL_TIME = "{}-{}-{}T{}:{}:00".format(YEAR, MONTH, DAY, HOUR, MINUTE)
ETHER_UNITS = 1e18
LIMIT = 200
MAX_OFFSET = 10000
SLEEP = 2
METHOD = "raritytools"
URL = "https://api.opensea.io/api/v1/events"
SHOW_INDIVIDUAL_MAPS = False
OPENSEA_API_KEY = "YOUR_API_KEY"

"""
Plot params
"""
plt.rcParams.update({"figure.facecolor": "white", "savefig.facecolor": "white"})

"""
Helper Functions
"""


def getOpenseaEvents(contract, offset, before_time):
    print(offset)
    querystring = {
        "event_type": "successful",
        "only_opensea": "false",
        "offset": offset,
        "limit": "200",
        "asset_contract_address": contract,
        "occurred_before": before_time,
    }

    headers = {"Accept": "application/json", "X-API-KEY": OPENSEA_API_KEY}

    response = requests.request("GET", URL, headers=headers, params=querystring)
    resJson = response.json()
    return resJson


def getMintEvents(contract, rarity_db):
    offset = 0
    data = getOpenseaEvents(contract, offset, REVEAL_TIME)
    events = data["asset_events"]
    sales_list = []
    before_time = REVEAL_TIME

    while len(data["asset_events"]) >= LIMIT and offset <= MAX_OFFSET:
        if offset >= MAX_OFFSET:
            old_before_time = events[-1]["transaction"]["timestamp"]
            datetime_before_time = datetime.datetime.strptime(
                old_before_time, "%Y-%m-%dT%H:%M:%S"
            )
            time_change = datetime.timedelta(minutes=TIME_DELTA)

            new_time = datetime_before_time + time_change
            before_time = new_time.strftime("%Y-%m-%dT%H:%M:%S")
            print("resetting offset from ", old_before_time, " to ", new_time)
            offset = 0
        else:
            offset = offset + LIMIT

        time.sleep(SLEEP)
        data = getOpenseaEvents(contract, offset, before_time)

        for event in data["asset_events"]:
            if event["transaction"] is not None:
                events.append(event)
            else:
                print("fail tx")

    for event in events:
        try:
            tokenId = int(event["asset"]["token_id"])
            sale = dict()
            sale["TOKEN_ID"] = tokenId
            sale["USER"] = event["transaction"]["from_account"]["address"]
            sale["DATE"] = event["created_date"]
            sale["RANK"] = int(RARITY_DB[RARITY_DB["TOKEN_ID"] == tokenId]["Rank"])
            sale["PRICE"] = float(event["total_price"]) / ETHER_UNITS
            sales_list.append(sale)
        except:
            continue

    return pd.DataFrame.from_dict(sales_list)

In [ ]:
"""
Generate Plot
"""

print(REVEAL_TIME)
RARITY_CSV = "../metadata/rarity_data/{}_{}.csv".format(COLLECTION, METHOD)
RARITY_DB = pd.read_csv(RARITY_CSV)


sales_df = getMintEvents(CONTRACT, RARITY_DB)
sales_df = sales_df.sort_values(by="RANK")
sales_df.to_csv("pre-reveal_sales/{}_pre-reveal_sales.csv".format(COLLECTION))

ax = sales_df.plot.scatter(
    x="TOKEN_ID",
    y="RANK",
    grid=True,
    alpha=0.25,
    title="{} - {}".format(COLLECTION, "Prereveal-Sales"),
    figsize=(14, 7),
)
ax.set_xlabel("Token ID")
ax.set_ylabel("Rarity Rank")